In [ ]:
import time
import numpy as np
import xarray as xr
from pathlib import Path

from pyPulses.utils import SweepMeasureProduct
from ez_data import DsetConnector, sqlite_to_xarray, xarray_to_sqlite

In [ ]:
VG_ = 0.0
def VG(x = None):
    global VG_
    if x is None: 
        return VG_
    VG_ = x

VB_ = 0.0
def VB(x = None):
    global VB_
    if x is None: 
        return VB_
    VB_ = x

def I():
    return np.random.normal(loc = 1.0, scale = 0.001)

def V():
    return np.random.normal(loc = 0.0, scale = 0.01)

In [ ]:
# Performance without any data saving

sweep = SweepMeasureProduct(
    coordinates    = [{'f': VG, 'name': 'VG', 'long_name': R"$V_G$", 'units': 'V'},
                      {'f': VB, 'name': 'VB', 'long_name': R"$V_B$", 'units': 'V'}],
    measurements   = [{'f': I, 'name': 'I', 'long_name': R"$I$", 'units': 'A'},
                      {'f': V, 'name': 'V', 'long_name': R"$V$", 'units': 'V'}],
    time_per_point = 0.01,
    axes           = (np.linspace(-1, 1, 50), np.linspace(-0.5, 0.5, 40)),
    retain_return  = False,
    plot_fields    = None,
    timestamp      = True
)

t0 = time.perf_counter()
sweep.run()
t1 = time.perf_counter()

print(f"Sweep finished in {t1 - t0:.3f} seconds")

In [ ]:
# Performance using DsetConnector

sweep = SweepMeasureProduct(
    coordinates    = [{'f': VG, 'name': 'VG', 'long_name': R"$V_G$", 'units': 'V'},
                      {'f': VB, 'name': 'VB', 'long_name': R"$V_B$", 'units': 'V'}],
    measurements   = [{'f': I, 'name': 'I', 'long_name': R"$I$", 'units': 'A'},
                      {'f': V, 'name': 'V', 'long_name': R"$V$", 'units': 'V'}],
    time_per_point = 0.01,
    axes           = (np.linspace(-1, 1, 50), np.linspace(-0.5, 0.5, 40)),
    retain_return  = False,
    plot_fields    = None,
    timestamp      = True
)

demo_path = Path("demo/demo_sweep")
connector = DsetConnector.from_param_sweep(sweep, demo_path)

sweep.post_callback = connector.add_point
t0 = time.perf_counter()
sweep.run()
connector.close()
t1 = time.perf_counter()

print(f"Sweep finished in {t1 - t0:.3f} seconds")

In [ ]:
# Performance using DsetConnector with buffering

sweep = SweepMeasureProduct(
    coordinates    = [{'f': VG, 'name': 'VG', 'long_name': R"$V_G$", 'units': 'V'},
                      {'f': VB, 'name': 'VB', 'long_name': R"$V_B$", 'units': 'V'}],
    measurements   = [{'f': I, 'name': 'I', 'long_name': R"$I$", 'units': 'A'},
                      {'f': V, 'name': 'V', 'long_name': R"$V$", 'units': 'V'}],
    time_per_point = 0.01,
    axes           = (np.linspace(-1, 1, 50), np.linspace(-0.5, 0.5, 40)),
    retain_return  = False,
    plot_fields    = None,
    timestamp      = True
)

demo_path = Path("demo/demo_sweep_buffered")
connector = DsetConnector.from_param_sweep(sweep, demo_path, use_buffer = True)

sweep.post_callback = connector.add_point
t0 = time.perf_counter()
sweep.run()
connector.close()
t1 = time.perf_counter()

print(f"Sweep finished in {t1 - t0:.3f} seconds")

In [ ]:
ds1 = sqlite_to_xarray("demo/demo_sweep")
ds2 = sqlite_to_xarray("demo/demo_sweep_buffered")
display(ds1)
display(ds2)

In [ ]:
# saving datasets
xarray_to_sqlite(ds2, "demo/demo_saved", overwrite=True)

In [ ]:
ds_saved = sqlite_to_xarray("demo/demo_saved")

In [ ]:
display(ds_saved)